In [2]:
from fuzzy_search.fuzzy_context_searcher import FuzzyContextSearcher
from fuzzy_search.fuzzy_phrase_searcher import FuzzyPhraseSearcher
from fuzzy_search.fuzzy_searcher import FuzzySearcher
from fuzzy_search.fuzzy_phrase_model import PhraseModel

In [6]:
config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}
fcs = FuzzyContextSearcher(config=config)
fps = FuzzyPhraseSearcher(config=config)

phrase = PhraseModel()

In [8]:
vs = ['PRASIDE Den Heere',
          'PRESIDE Den Heere',
          'P R A E S I D E Den Heere',
          'PRAESIDE Den Heere',
          'P R A S I D E Den Heere',
          'P R E S I D E Den Heere',
          'P R AE S I D E Den Heere',
          'DR AS 1D E Den Heere',
          'PR ASL DE Den Heere',
          'PR A31 DE; Den Heere',
          'BR JE 3.1. DE, Den Heere']
vs = vs + ekwz['PRAS']
pvs = ['PRASENTIBUS',
                'PRESENTIBUS',
                'P R A E S E N T I B U S',
                'P RAE SE N TI B U S',
                'PRA&SENTIBUS']
variants = [{'phrase': 'PRAESIDE Den Heere', 'variants': vs},
            {'phrase':'PRAESENTIBUS', "variants":ekwz['PRASENTIBUS']}]
phrase_model = PhraseModel(model=variants)
phrase_model.json

[{'phrase': 'PRAESIDE Den Heere',
  'variants': ['BRAS',
   'P R A E S I D E Den Heere',
   'BR JE 3.1. DE, Den Heere',
   'PRASIDE Den Heere',
   'DR AS 1D E Den Heere',
   'Pros',
   'PR&S',
   'P R AE S I D E Den Heere',
   'PRA&S',
   'P R A S I D E Den Heere',
   'PR&AS',
   'PRAES',
   'PRA',
   'PR A31 DE; Den Heere',
   'PRESIDE Den Heere',
   'P R E S I D E Den Heere',
   'PR ASL DE Den Heere',
   'PRES',
   'PREAS',
   'PRAESIDE Den Heere'],
  'custom': {'phrase': 'PRAESIDE Den Heere',
   'variants': ['PRASIDE Den Heere',
    'PRESIDE Den Heere',
    'P R A E S I D E Den Heere',
    'PRAESIDE Den Heere',
    'P R A S I D E Den Heere',
    'P R E S I D E Den Heere',
    'P R AE S I D E Den Heere',
    'DR AS 1D E Den Heere',
    'PR ASL DE Den Heere',
    'PR A31 DE; Den Heere',
    'BR JE 3.1. DE, Den Heere',
    'PRAES',
    'PR&S',
    'PREAS',
    'Pros',
    'PRES',
    'PR&AS',
    'PRA&S',
    'PRA',
    'BRAS']}},
 {'phrase': 'PRAESENTIBUS',
  'variants': ['PRASELNTIBU

In [9]:
from republic.data.delegate_database import ekwz
# create a list of domain keywords and phrases
# variants = [{'phrase': k, 'variants': v} for k, v in ekwz.items()]
# # create a PhraseModel object from the domain phrases
#phrase_model = PhraseModel(model=variants,)
#phrase_model.add_phrases(ekwz.keys())
# phrase_model.add_variants(variants)
fps.index_phrase_model(phrase_model=phrase_model)
fcs.index_phrase_model(phrase_model=phrase_model)

In [11]:
p = ekwz['PRASENTIBUS']
pvs = ['PRASENTIBUS',
                'PRESENTIBUS',
                'P R A E S E N T I B U S',
                'P RAE SE N TI B U S',
                'PRA&SENTIBUS']
p+pvs

['Prasentibus',
 'PRESENTIBUS',
 'TIBÚS',
 'TIBDUS',
 'TIEBUS',
 'PRA&ASEHNTIBUS',
 'TIBÚUS',
 'TIDUS',
 'TLBUS',
 'TIBUS',
 'TIBÜS',
 'TIBDUDS',
 'PR&SENTIBUS',
 'ENTIBUS',
 'PRASELNTIBUS',
 'PRASENTIBUS',
 'PRESENTIBUS',
 'P R A E S E N T I B U S',
 'P RAE SE N TI B U S',
 'PRA&SENTIBUS']

In [12]:
from fuzzy_search.fuzzy_string import score_levenshtein_similarity_ratio

In [13]:
res = fps.find_matches(text="PRAESIDE Den Heere", include_variants=True, use_word_boundaries=False, allow_overlapping_matches=True)
# for item in res:
#     print(item.score_levenshtein_similarity())
from scipy.special import softmax
from numpy import argmax
def score_match(match):
    result = sum([match.levenshtein_similarity, match.character_overlap, match.ngram_overlap, score_levenshtein_similarity_ratio(str(match.variant.exact_string),str(match.phrase.exact_string))])
    return result

b = argmax(softmax([score_match(i) for i in res]))
res[b]

PhraseMatch(phrase: "PRAESIDE Den Heere", variant: "PRAESIDE Den Heere",string: "PRAESIDE Den Heere", offset: 0)

In [16]:
res

[]

In [23]:
from republic.helper.utils import *
res = fps.find_matches(text="ie Veucris den 5. Januaris 1725. PR&ASIDE, Den Heere Bentinck. PRASENTIEBUS, De Heeren Jan Welderen , van Dam, Torck , met een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt. Van Maasdam , vanden Boeizelaar , Raadtpenfionaris van Hoornbeeck , met een extraordinaris Gedeputeerde uyt de Provincie van Hollandt ende Welt-Vrieslandt. Velters, Ockere , Noey; van Hoorn , met een extraordinaris Gedeputeerde uyt de Provincie van Zeelandt. Van Renswoude , van Voor{t. Van Schwartzenbergh, vander Waayen, Vegilin Van I{elmuden. Van Iddekinge ‚ van Tamminga."
, include_variants=True, use_word_boundaries=False, allow_overlapping_matches=True)
mx = max([score_match(m) for m in res])
[m for m in res if score_match(m)==mx]
r = res[0]
r.end - r.offset
ranges = [set(range(r.offset, r.end)) for r in res]
p = pairwise
setpair = []
for pair in p(ranges):
    pair = list(pair)
    pair.sort()
    if pair[0].intersection(pair[1]):
        if pair[0].issubset(pair[1]):
            setpair.append(pair[1])
setpair  

[{33, 34, 35, 36, 37}, {63, 64, 65, 66}]

In [25]:
mx = max(ranges, key=len)

ranges.pop(ranges.index(mx))

groupby(ranges, key=lambda x:set(x) & mx)
Out[23]: <itertools.groupby at 0x109ead9b0>

In [24]: list(groupby(ranges, key=lambda x:set(x) & mx))
Out[24]:
[(set(), <itertools._grouper at 0x109cf5b90>),
 ({12, 13, 14, 15, 16}, <itertools._grouper at 0x109af6150>),
 ({12, 13, 14, 15, 16, 17}, <itertools._grouper at 0x109af6190>),
 ({12, 13, 14, 15}, <itertools._grouper at 0x109af6c50>)]

In [25]: for item in ranges:
    ...:     print(set(item) & set(mx)
    ...:     )
    ...:
set()
set()
{12, 13, 14, 15, 16}
{12, 13, 14, 15, 16}
{12, 13, 14, 15, 16, 17}
{12, 13, 14, 15}

In [26]: list(groupby(ranges, key=lambda x:set(x) & set(mx)))
Out[26]:
[(set(), <itertools._grouper at 0x109eb3f50>),
 ({12, 13, 14, 15, 16}, <itertools._grouper at 0x109c0fb50>),
 ({12, 13, 14, 15, 16, 17}, <itertools._grouper at 0x109c0f6d0>),
 ({12, 13, 14, 15}, <itertools._grouper at 0x109c0ff50>)]

In [27]: for item in enumerate(ranges):
    ...:     if set(item[1]) & set(mx):
    ...:         ranges.pop(item[0])
    ...:
    ...:

In [28]: ranges
Out[28]: [{0, 1, 2, 3}, {0, 1, 2, 3}, {12, 13, 14, 15, 16}, {12, 13, 14, 15}]

In [29]: mx
Out[29]: {12, 13, 14, 15, 16, 17}

In [30]: for item in enumerate(ranges):
    ...:     if set(item[1]) & set(mx):
    ...:         ranges.pop(item[0])
    ...:
    ...:

In [31]: ranges
Out[31]: [{0, 1, 2, 3}, {0, 1, 2, 3}, {12, 13, 14, 15}]

In [32]: for item in enumerate(ranges):
    ...:     if set(item[1]) & set(mx):
    ...:         ranges.pop(item[0])
    ...:
    ...:

In [33]: ranges
Out[33]: [{0, 1, 2, 3}, {0, 1, 2, 3}]

In [34]: mx = max(ranges, len)
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-34-e37f3554337d> in <module>
----> 1 mx = max(ranges, len)

TypeError: '>' not supported between instances of 'builtin_function_or_method' and 'list'

In [35]: mx = max(ranges, len)

SyntaxError: invalid syntax (<ipython-input-25-0c157bab0b15>, line 6)